In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
from spark_plot import mpl
mpl.set_defaults()

In [ ]:
spark = (SparkSession
    .builder
    .appName("nycflights-hist-dev")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .getOrCreate()
)

In [ ]:
from nycflights13 import flights as flights_pd

In [ ]:
flights = spark.createDataFrame(flights_pd)

In [ ]:
flights.persist()

In [ ]:
from spark_plot import utils
from pyspark.ml.feature import Bucketizer

In [ ]:
column = "distance"
bins = 30

In [ ]:
data = flights[[column]]

In [ ]:
buckets = utils.spark_buckets(data, column, bins=bins)

In [ ]:
bucketizer = Bucketizer(splits=buckets, inputCol=column, outputCol="bucket")
buckets_df = bucketizer.setHandleInvalid("error").transform(data)

In [ ]:
histogram = buckets_df.groupby("bucket").agg(F.count(column).alias("count"))

In [ ]:
histogram = histogram.orderBy("bucket", ascending=True)

In [ ]:
histogram.show()

In [ ]:
hist_pd = histogram.toPandas()
hist_pd

In [ ]:
fill_buckets = pd.DataFrame(columns=["bucket"])

In [ ]:
fill_buckets["bucket"] = np.arange(len(buckets))

In [ ]:
fill_buckets = fill_buckets.merge(hist_pd, on="bucket", how='left')

In [ ]:
weights = fill_buckets["count"]

In [ ]:
len(weights)

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.hist(buckets, len(buckets), weights=weights)

## Compare to Pandas

In [ ]:
flights_pd.hist(column, bins=40, figsize=(10, 10))